In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import pairwise_distances_argmin_min
from skimage.color import deltaE_ciede2000, rgb2lab, rgb2gray

# Preprocessing

In [2]:
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [3]:
df_to_trimm=pd.read_csv('/content/makeupDataset_final_sp2.csv')
df_to_trimm

,product category,product type,product name,skin type,formulation,finish,coverage,hydration,oil free,vegan,...,gluten free,cruelty free,product color,hexa,brand,price,product link,ingeredient highlights,ratings,review
0,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Skye ST0.5 - Extra light with neutral undertones,#f4d8b4,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation blends seamlessly into my ski..."
1,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Rendezvous ST1 - Extra light with cool undertones,#f5d3b7,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""I love the dewy finish! It makes my skin look..."
2,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Tulum ST2 - Very light with warm undertones,#f1d3a9,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""The coverage is buildable, which is perfect f..."
3,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Sombrio ST2.5 - Very light with warm olive und...,#fed8aa,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation did not cause any irritation,..."
4,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Balos ST3 - Very light with neutral cool under...,#edc79e,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""It's perfect for oily skin! I stay shine-free..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,Cheek,Blush,Hydromaniac Dewy Liquid Blush,all,Liquid,Radiant,Medium,yes,yes,yes,...,yes,yes,Red Flag - red orange,#7f251d,Urban Decay,22.0,https://www.sephora.com/product/urban-decay-hy...,Kombucha: Detoxifies and nourishes skin.,4.4,"Bought this for my sister, but she found it ch..."
1718,Cheek,Blush,Hydromaniac Dewy Liquid Blush,all,Liquid,Radiant,Medium,yes,yes,yes,...,yes,yes,Rebellious - burnt terracota,#a04d42,Urban Decay,22.0,https://www.sephora.com/product/urban-decay-hy...,Kombucha: Detoxifies and nourishes skin.,4.4,This blusher gives me the perfect rosy glow an...
1719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## function that drops empty rows

In [4]:
def drop_rows_after(df, row_number=1719):
    return df.iloc[:row_number]  # Keep only rows before row_number

df = drop_rows_after(df_to_trimm, row_number=1719)
df

,product category,product type,product name,skin type,formulation,finish,coverage,hydration,oil free,vegan,...,gluten free,cruelty free,product color,hexa,brand,price,product link,ingeredient highlights,ratings,review
0,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Skye ST0.5 - Extra light with neutral undertones,#f4d8b4,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation blends seamlessly into my ski..."
1,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Rendezvous ST1 - Extra light with cool undertones,#f5d3b7,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""I love the dewy finish! It makes my skin look..."
2,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Tulum ST2 - Very light with warm undertones,#f1d3a9,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""The coverage is buildable, which is perfect f..."
3,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Sombrio ST2.5 - Very light with warm olive und...,#fed8aa,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation did not cause any irritation,..."
4,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,yes,yes,Balos ST3 - Very light with neutral cool under...,#edc79e,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""It's perfect for oily skin! I stay shine-free..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,Cheek,Blush,Hydromaniac Dewy Liquid Blush,all,Liquid,Radiant,Medium,yes,yes,yes,...,yes,yes,Drippin' - bubblegum pink,#dc3c54,Urban Decay,22.0,https://www.sephora.com/product/urban-decay-hy...,Kombucha: Detoxifies and nourishes skin.,4.4,"Great for sensitive skin with a soft, natural ..."
1715,Cheek,Blush,Hydromaniac Dewy Liquid Blush,all,Liquid,Radiant,Medium,yes,yes,yes,...,yes,yes,Unhinged - berry mauve,#c83234,Urban Decay,22.0,https://www.sephora.com/product/urban-decay-hy...,Kombucha: Detoxifies and nourishes skin.,4.4,"The color is pretty, but it doesn’t blend well..."
1716,Cheek,Blush,Hydromaniac Dewy Liquid Blush,all,Liquid,Radiant,Medium,yes,yes,yes,...,yes,yes,Obsessed - coral peach,#a10b17,Urban Decay,22.0,https://www.sephora.com/product/urban-decay-hy...,Kombucha: Detoxifies and nourishes skin.,4.4,The creamy texture and strong pigmentation mak...
1717,Cheek,Blush,Hydromaniac Dewy Liquid Blush,all,Liquid,Radiant,Medium,yes,yes,yes,...,yes,yes,Red Flag - red orange,#7f251d,Urban Decay,22.0,https://www.sephora.com/product/urban-decay-hy...,Kombucha: Detoxifies and nourishes skin.,4.4,"Bought this for my sister, but she found it ch..."


## Assign the product color to our scale and add it as new feature

In [5]:
# new scale to run
new_scale = {
    'Class1': (233, 216, 200), #E9D8C8
    'Class2': (226, 189, 174), #E2BDAE
    'Class3': (211, 185, 152), #D3B998
    'Class4': (198, 158, 118), #C69E76
    'Class5': (193, 132, 99),  #C18463
    'Class6': (145, 114, 82),  #917252
    'Class7': (109, 74, 55),   #6D4A37
    'Class8': (62, 44, 30)     #3E2C1E
}


In [6]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.strip('[]').split(',')[0].replace("'", "")
    if len(hex_color) == 7 and hex_color[0] == '#':
        try:
            r = int(hex_color[1:3], 16)
            g = int(hex_color[3:5], 16)
            b = int(hex_color[5:], 16)
            return (r, g, b)
        except ValueError:
            return None
    return None

In [7]:
def find_closest_class(rgb_color, product_category):
    if product_category == 'Face':
        # Use deltaE_ciede2000 to find closest class
        lab_color = rgb2lab(np.array([[rgb_color]]) / 255.0)[0][0]
        scale_colors = {key: rgb2lab(np.array([[color]]) / 255.0)[0][0] for key, color in new_scale.items()}

        min_distance = float('inf')
        closest_class = None

        for class_name, class_lab_color in scale_colors.items():
            distance = deltaE_ciede2000(lab_color, class_lab_color)
            if distance < min_distance:
                min_distance = distance
                closest_class = class_name

        return closest_class
    elif product_category == 'Cheek':
        # Convert RGB to grayscale and find closest gray value
        gray_value = rgb2gray(np.array([[rgb_color]]) / 255.0)[0][0]
        scale_colors_gray = {key: rgb2gray(np.array([[color]]) / 255.0)[0][0] for key, color in new_scale.items()}

        min_distance = float('inf')
        closest_class = None

        for class_name, class_gray_value in scale_colors_gray.items():
            distance = abs(gray_value - class_gray_value)
            if distance < min_distance:
                min_distance = distance
                closest_class = class_name

        return closest_class
    else:
        return None

In [8]:
df['hexa '] = df['hexa '].astype(str)

In [9]:
df['rgb'] = df['hexa '].apply(hex_to_rgb)

In [10]:
df['class color'] = df.apply(lambda row: find_closest_class(row['rgb'], row['product category']) if not pd.isna(row['rgb']) else None, axis=1)
# Use the 'rgb' column which contains the output of hex_to_rgb, instead of 'hexa '

In [11]:
# Clean the 'product category' column just to add the class color feature
df['product category'] = df['product category'].str.strip()

print(df['product category'].unique())

['Face' 'Cheek']


In [12]:
df.head(5)

,product category,product type,product name,skin type,formulation,finish,coverage,hydration,oil free,vegan,...,product color,hexa,brand,price,product link,ingeredient highlights,ratings,review,rgb,class color
0,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Skye ST0.5 - Extra light with neutral undertones,#f4d8b4,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation blends seamlessly into my ski...","(244, 216, 180)",Class1
1,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Rendezvous ST1 - Extra light with cool undertones,#f5d3b7,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""I love the dewy finish! It makes my skin look...","(245, 211, 183)",Class1
2,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Tulum ST2 - Very light with warm undertones,#f1d3a9,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""The coverage is buildable, which is perfect f...","(241, 211, 169)",Class3
3,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Sombrio ST2.5 - Very light with warm olive und...,#fed8aa,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation did not cause any irritation,...","(254, 216, 170)",Class3
4,Face,Foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Balos ST3 - Very light with neutral cool under...,#edc79e,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""It's perfect for oily skin! I stay shine-free...","(237, 199, 158)",Class3


##Cleaning the feature values

This cleaning process includes:

- Converting values to lowercase.
- Stripping extra spaces.
- Removing duplicates.
- Correcting specific values like 'bawder' to 'powder', 'liqued' to 'liquid', and 'matt' to 'matte'.

In [13]:
# Check the column names to ensure there are no hidden spaces or special characters
print("Column names before cleaning:", df.columns)

# Cleaning column names (removing extra spaces or special characters)
df.columns = df.columns.str.strip()

# Verify if the 'hydration' column exists after cleaning
print("Column names after cleaning:", df.columns)

Column names before cleaning: Index(['product category', 'product type', 'product name', 'skin type',
       'formulation', 'finish', 'coverage', ' hydration', 'oil free', 'vegan',
       'fragrance free', 'gluten free', 'cruelty free', 'product color',
       'hexa ', 'brand', 'price', 'product link', 'ingeredient highlights',
       'ratings', 'review', 'rgb', 'class color'],
      dtype='object')
Column names after cleaning: Index(['product category', 'product type', 'product name', 'skin type',
       'formulation', 'finish', 'coverage', 'hydration', 'oil free', 'vegan',
       'fragrance free', 'gluten free', 'cruelty free', 'product color',
       'hexa', 'brand', 'price', 'product link', 'ingeredient highlights',
       'ratings', 'review', 'rgb', 'class color'],
      dtype='object')


In [14]:
# Let's clean all the relevant features first
features = ['product category', 'product type', 'skin type', 'formulation', 'finish', 'coverage', 'hydration', 'oil free', 'vegan', 'fragrance free', 'gluten free', 'cruelty free', 'class color']

# Cleaning feature values for all the relevant columns
df[features] = df[features].apply(lambda col: col.str.strip().replace(r'\n', '', regex=True))

# Now let's get the unique values for each feature
unique_values = {feature: df[feature].unique().tolist() for feature in features}
unique_values


{'product category': ['Face', 'Cheek'],
 'product type': ['Foundation', 'Concealer', 'Blush'],
 'skin type': ['all'],
 'formulation': ['liquid',
  'cream',
  'stick',
  'Liquid',
  'Crème & Powder',
  'Cream',
  'Stick',
  'Powder',
  'liqued',
  'bawder'],
 'finish': ['radiant',
  'matte',
  'natural',
  'natural and radiant',
  'Radiant',
  'Natural',
  'Matt',
  'Shimmer',
  'shimmer',
  'Satin',
  'Matte',
  nan,
  'Matte\xa0+ Shimmer',
  'matte+highlight'],
 'coverage': ['light',
  'full',
  'medium',
  'Full',
  'Medium',
  'Light',
  nan,
  'buildable'],
 'hydration': ['yes', 'no'],
 'oil free': ['yes', 'no'],
 'vegan': ['yes', 'no'],
 'fragrance free': ['yes', 'no', nan],
 'gluten free': ['yes', 'no'],
 'cruelty free': ['yes', 'no'],
 'class color': ['Class1',
  'Class3',
  'Class4',
  'Class6',
  'Class5',
  'Class7',
  'Class8',
  'Class2',
  None]}

In [15]:
def clean_values(values):
    if isinstance(values, list):
        # Convert all values to lowercase and strip whitespaces
        cleaned = [str(val).strip().lower() for val in values if pd.notna(val)]
        # Remove duplicates by converting to a set and then back to a list
        cleaned = list(set(cleaned))
        return cleaned
    return values

# Applying the cleaning function to the unique values extracted from the DataFrame
cleaned_unique_values = {feature: clean_values(values) for feature, values in unique_values.items()}

# Create a mapping for corrections
corrections = {
    'bawder': 'powder',
    'liqued': 'liquid',
    'matt': 'matte'
}

# Function to apply the corrections
def apply_corrections(values, corrections):
    return [corrections.get(val, val) for val in values]

# Apply corrections after cleaning
cleaned_unique_values['formulation'] = apply_corrections(cleaned_unique_values['formulation'], corrections)
cleaned_unique_values['finish'] = apply_corrections(cleaned_unique_values['finish'], corrections)

# Now we need to update the DataFrame with the cleaned values

# Loop over the features and update the DataFrame columns with the cleaned values
for feature in cleaned_unique_values.keys():
    # Get the original column from the DataFrame
    # Check if the element is a string before applying strip and lower
    df[feature] = df[feature].apply(lambda x: corrections.get(str(x).strip().lower(), str(x).strip().lower()) if isinstance(x, str) else x)

# Display the cleaned DataFrame
df.head()

,product category,product type,product name,skin type,formulation,finish,coverage,hydration,oil free,vegan,...,product color,hexa,brand,price,product link,ingeredient highlights,ratings,review,rgb,class color
0,face,foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Skye ST0.5 - Extra light with neutral undertones,#f4d8b4,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation blends seamlessly into my ski...","(244, 216, 180)",class1
1,face,foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Rendezvous ST1 - Extra light with cool undertones,#f5d3b7,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""I love the dewy finish! It makes my skin look...","(245, 211, 183)",class1
2,face,foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Tulum ST2 - Very light with warm undertones,#f1d3a9,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""The coverage is buildable, which is perfect f...","(241, 211, 169)",class3
3,face,foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Sombrio ST2.5 - Very light with warm olive und...,#fed8aa,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""This foundation did not cause any irritation,...","(254, 216, 170)",class3
4,face,foundation,Super Serum Skin Tint SPF 40 Skincare Foundation,all,liquid,radiant,light,yes,yes,yes,...,Balos ST3 - Very light with neutral cool under...,#edc79e,ILIA,48.0,https://www.sephora.com/product/ilia-super-ser...,"Clean at Sephora, Community Favorite, Cruelty-...",5.6,"""It's perfect for oily skin! I stay shine-free...","(237, 199, 158)",class3


## Convert 'class color' column to integers

In [16]:
# Inspect the original values in the 'class color' column
original_class_colors = df['class color'].unique()
print("Original Class Colors:", original_class_colors)

# Clean the 'class color' column (strip spaces, convert to lowercase)
df['class color'] = df['class color'].str.strip().str.lower()

# Redefine the mapping to match cleaned values (lowercase)
class_color_mapping = {
    'class1': 1,
    'class2': 2,
    'class3': 3,
    'class4': 4,
    'class5': 5,
    'class6': 6,
    'class7': 7,
    'class8': 8,
    None: 0
}

# Apply the mapping to the cleaned 'class color' column
df['class color'] = df['class color'].map(class_color_mapping)

# Convert the column to integers, handling NaNs
df['class color'] = pd.to_numeric(df['class color'], errors='coerce').fillna(0).astype(int)

# Check the unique values again after cleaning
unique_class_colors_cleaned = df['class color'].unique()
print("Unique Class Colors After Cleaning:", unique_class_colors_cleaned)


Original Class Colors: ['class1' 'class3' 'class4' 'class6' 'class5' 'class7' 'class8' 'class2'
 None]
Unique Class Colors After Cleaning: [1 3 4 6 5 7 8 2 0]


##Saving the data after Claening

In [19]:
# Save cleaned dataframe
df_cleaned = df.copy()

# Save to a new CSV file
df_cleaned.to_csv('makeup_knowledge_r_data.csv', index=False)

print("Dataset cleaned and saved as 'df_cleaned.csv'.")

Dataset cleaned and saved as 'df_cleaned.csv'.


#Recommender

In [62]:
user_preferences_encoded = {
    'product type': 'blush',
    'skin type': 'all',
    'formulation': '',
    'finish': '',
    'coverage': '',
    'hydration': '',
    'oil free': '',
    'vegan': '',
    'fragrance free': '',
    'gluten free': '',
    'cruelty free': '',
    'class color': 3
}

user_pref_df = pd.DataFrame([user_preferences_encoded])

##First approach

In [63]:
def recommend_based_on_preferences_encoded(df, preferences, n_recommendations=3, max_duplicates=2):
    # Define default values for preferences
    default_values = {
        'skin type': 'all',
        'formulation': 'liquid',
        'finish': 'radiant',
        'hydration': 'no',
        'oil free': 'no',
        'vegan': 'no',
        'fragrance free': 'no',
        'gluten free': 'no',
        'cruelty free': 'no'
    }

    # Get all unique values for 'class color', 'product type', and 'coverage' from the DataFrame
    all_class_colors = df['class color'].unique().tolist()
    all_product_types = df['product type'].unique().tolist()
    all_coverages = df['coverage'].unique().tolist()

    # Update default values to include the first value of all class color, product type, and coverage
    default_values['class color'] = all_class_colors[0] if all_class_colors else 1
    default_values['product type'] = all_product_types[0] if all_product_types else 'Foundation'
    default_values['coverage'] = all_coverages[0] if all_coverages else 'medium'

    # Update the user preferences with default values if any are missing
    for feature, default in default_values.items():
        if preferences.get(feature, '') == '':
            preferences[feature] = default


    # Check if the user has provided a specific product type ------------------------------------------------------------------------------------
    selected_product_type = preferences.get('product_type', '').lower()

    # Handle Concealer logic with numerical class color: Get the class color and find two degrees below
    if selected_product_type == 'concealer':
        selected_class_color = preferences.get('class color', None)
        if selected_class_color is not None:
            selected_class_color = int(selected_class_color)
            # Include the selected class color and two levels below
            class_colors_to_include = [selected_class_color, max(1, selected_class_color - 1), max(1, selected_class_color - 2)]

            # Filter DataFrame by selected product type ('Concealer') and the relevant class colors
            df_filtered = df[(df['class color'].isin(class_colors_to_include)) & (df['product type'] == 'concealer')]
        else:
            # If no class color is specified, filter only by 'Concealer'
            df_filtered = df[df['product type'] == 'concealer']
    else:
        # Filter by the user-selected product type; if none provided, use all available types
        df_filtered = df[df['product type'] == selected_product_type] if selected_product_type else df

    # One-Hot Encoding categorical features ----------------------------------------------------------------------------------------------------
    features = ['product type', 'skin type', 'formulation', 'finish', 'coverage', 'hydration', 'oil free', 'vegan', 'fragrance free', 'gluten free', 'cruelty free', 'class color']
    combined_df = df_filtered[features]
    encoder = OneHotEncoder()
    #encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # handle_unknown='ignore' will prevent errors if unknown categories are encountered during prediction
    encoded_features = encoder.fit_transform(combined_df)

    # Encode the user preferences into a one-hot vector
    user_pref_df = pd.DataFrame([preferences])
    encoded_user_pref = encoder.transform(user_pref_df[features])

    # Compute cosine similarity between the user's preferences and the product data ------------------------------------------------------------
    cosine_sim_user = cosine_similarity(encoded_user_pref, encoded_features)

    # Get the top indices based on similarity
    top_indices = cosine_sim_user[0].argsort()[-n_recommendations:][::-1]

    # Normalize similarity scores to percentages (0-100)
    similarity_percentages = (cosine_sim_user[0][top_indices] * 100).round(2)

    # Filter the top recommendations and allow only a maximum number of duplicates -------------------------------------------------------------
    recommended_products = df_filtered.iloc[top_indices]

    # Control duplicates by allowing 'max_duplicates' for each product name
    recommended_products['product_rank'] = recommended_products.groupby('product name').cumcount() + 1
    recommended_products = recommended_products[recommended_products['product_rank'] <= max_duplicates].drop('product_rank', axis=1)

    # Add the similarity percentage to the final DataFrame
    recommended_products['similarity percentage'] = similarity_percentages[:len(recommended_products)] # Limit similarity percentages to the DataFrame length

    # Return the most similar products based on the filtered and encoded preferences
    return recommended_products[['product type', 'product category', 'product name', 'brand', 'price', 'ratings', 'ingeredient highlights', 'class color', 'similarity percentage']]

In [74]:
user_preferences_encoded = {
    'product type': 'blush',
    'skin type': 'all',
    'formulation': 'liquid',
    'finish': 'matte',
    'coverage': 'full',
    'hydration': 'yes',
    'oil free': 'yes',
    'vegan': 'yes',
    'fragrance free': 'no',
    'gluten free': 'yes',
    'cruelty free': 'yes',
    'class color': 2
}

user_pref_df = pd.DataFrame([user_preferences_encoded])

In [75]:
recommendations = recommend_based_on_preferences_encoded(df_cleaned, user_preferences_encoded,n_recommendations=4 ,max_duplicates=4)
recommendations

,product type,product category,product name,brand,price,ratings,ingeredient highlights,class color,similarity percentage
38,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",2,83.33
45,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",2,83.33
839,concealer,face,Pro Filt’r Instant Retouch Longwear Liquid Con...,Fenty Beauty by Rihanna,30.0,2.2,"Community Favorite, Vegan, Full Coverage, Matt...",2,75.00
880,concealer,face,Pro Filt’r Instant Retouch Longwear Liquid Con...,Fenty Beauty by Rihanna,30.0,2.2,"Community Favorite, Vegan, Full Coverage, Matt...",2,75.00


##Second approach

In [76]:
def recommendation2(gen_df, user_preferences_encoded, n_recommendations=4, max_duplicates=4, percentile=0.8):
    # Step a: Update user preferences with default values if any are missing
    default_values = {
        'skin type': 'all',
        'formulation': 'liquid',
        'finish': 'radiant',
        'hydration': 'no',
        'oil free': 'no',
        'vegan': 'no',
        'fragrance free': 'no',
        'gluten free': 'no',
        'cruelty free': 'no'
    }

    # Get all unique values for 'class color', 'product type', and 'coverage' from the DataFrame
    all_class_colors = gen_df['class color'].unique().tolist()
    all_product_types = gen_df['product type'].unique().tolist()
    all_coverages = gen_df['coverage'].unique().tolist()

    # Update default values to include the first value of all class color, product type, and coverage
    default_values['class color'] = all_class_colors[0] if all_class_colors else 1
    default_values['product type'] = all_product_types[0] if all_product_types else 'Foundation'
    default_values['coverage'] = all_coverages[0] if all_coverages else 'medium'

    # Update user preferences with default values if any are missing
    for feature, default in default_values.items():
        if user_preferences_encoded.get(feature, '') == '':
            user_preferences_encoded[feature] = default

    # Step b: Extract all makeup products that match the conditions set by the user
    selected_product_type = user_preferences_encoded.get('product_type', '').lower()

    if selected_product_type == 'concealer':
        selected_class_color = user_preferences_encoded.get('class color', None)
        if selected_class_color is not None:
            selected_class_color = int(selected_class_color)
            class_colors_to_include = [selected_class_color, max(1, selected_class_color - 1), max(1, selected_class_color - 2)]
            df_filtered = gen_df[(gen_df['class color'].isin(class_colors_to_include)) & (gen_df['product type'] == 'concealer')]
        else:
            df_filtered = gen_df[gen_df['product type'] == 'concealer']
    else:
        df_filtered = gen_df[gen_df['product type'] == selected_product_type] if selected_product_type else gen_df

    # Step c: Calculate the values of m and C for only these makeup products and proceed to build the chart as in the previous section
    features = ['product type', 'skin type', 'formulation', 'finish', 'coverage', 'hydration', 'oil free', 'vegan', 'fragrance free', 'gluten free', 'cruelty free', 'class color']
    combined_df = df_filtered[features]
    encoder = OneHotEncoder()
    encoded_features = encoder.fit_transform(combined_df)

    user_pref_df = pd.DataFrame([user_preferences_encoded])
    encoded_user_pref = encoder.transform(user_pref_df[features])

    cosine_sim_user = cosine_similarity(encoded_user_pref, encoded_features)
    top_indices = cosine_sim_user[0].argsort()[-n_recommendations:][::-1]

    recommended_products = df_filtered.iloc[top_indices]
    recommended_products['similarity percentage'] = (cosine_sim_user[0][top_indices] * 100).round(2)

    # Control duplicates by allowing 'max_duplicates' for each product name
    recommended_products['product_rank'] = recommended_products.groupby('product name').cumcount() + 1
    recommended_products = recommended_products[recommended_products['product_rank'] <= max_duplicates].drop('product_rank', axis=1)

    return recommended_products[['product type', 'product category', 'product name', 'brand', 'price', 'ratings', 'ingeredient highlights', 'class color', 'similarity percentage']]


In [79]:
recommendations2 = recommendation2(df_cleaned, user_preferences_encoded, n_recommendations=4, max_duplicates=4)
recommendations2

,product type,product category,product name,brand,price,ratings,ingeredient highlights,class color,similarity percentage
38,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",2,83.33
45,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",2,83.33
839,concealer,face,Pro Filt’r Instant Retouch Longwear Liquid Con...,Fenty Beauty by Rihanna,30.0,2.2,"Community Favorite, Vegan, Full Coverage, Matt...",2,75.00
880,concealer,face,Pro Filt’r Instant Retouch Longwear Liquid Con...,Fenty Beauty by Rihanna,30.0,2.2,"Community Favorite, Vegan, Full Coverage, Matt...",2,75.00


In [82]:
user_preferences_encoded = {
    'product type': 'foundation',
    'skin type': 'all',
    'formulation': 'cream',
    'finish': 'matte',
    'coverage': 'full',
    'hydration': 'yes',
    'oil free': 'yes',
    'vegan': 'yes',
    'fragrance free': 'yes',
    'gluten free': 'yes',
    'cruelty free': 'yes',
    'class color': 3
}

user_pref_df = pd.DataFrame([user_preferences_encoded])

In [84]:
recommendations2 = recommendation2(df_cleaned, user_preferences_encoded, n_recommendations=10, max_duplicates=10)
recommendations2

,product type,product category,product name,brand,price,ratings,ingeredient highlights,class color,similarity percentage
33,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
39,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
46,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
42,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
41,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
43,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
37,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",3,91.67
38,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",2,83.33
40,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",1,83.33
36,foundation,face,Pro Filt’r Soft Matte Longwear Liquid Foundation,Fenty beauty,40.0,17.3,"Aqua/Water/Eau, Dimethicone, Talc, PEG-10 Dime...",1,83.33


In [88]:
user_preferences_encoded = {
    'product type': 'concealer',
    'skin type': 'all',
    'formulation': 'stick',
    'finish': 'matte',
    'coverage': 'full',
    'hydration': 'no',
    'oil free': 'no',
    'vegan': 'yes',
    'fragrance free': 'no',
    'gluten free': 'yes',
    'cruelty free': 'yes',
    'class color': 1
}

user_pref_df = pd.DataFrame([user_preferences_encoded])

In [89]:
recommendations2 = recommendation2(df_cleaned, user_preferences_encoded, n_recommendations=4, max_duplicates=4)
recommendations2

,product type,product category,product name,brand,price,ratings,ingeredient highlights,class color,similarity percentage
731,concealer,face,FauxFilter Matte Buildable Coverage Waterproof...,HUDA BEAUTY,31.0,5.0,"Vegan, Liquid Formula, Full Coverage, Matte Fi...",1,83.33
728,concealer,face,FauxFilter Matte Buildable Coverage Waterproof...,HUDA BEAUTY,31.0,5.0,"Vegan, Liquid Formula, Full Coverage, Matte Fi...",1,83.33
727,concealer,face,FauxFilter Matte Buildable Coverage Waterproof...,HUDA BEAUTY,31.0,5.0,"Vegan, Liquid Formula, Full Coverage, Matte Fi...",1,83.33
726,concealer,face,FauxFilter Matte Buildable Coverage Waterproof...,HUDA BEAUTY,31.0,5.0,"Vegan, Liquid Formula, Full Coverage, Matte Fi...",1,83.33


##The end
      .-""""-.
     / -    - \
    |  O    O  |
    |     >    |
     \  \__/  /
      `-.....-`
                /
        /|\\   /
        / | \\/
          |
        / \\
        /   \
